<a href="https://colab.research.google.com/github/eyaler/constrained/blob/main/density/ted_multilingual_counter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install PyICU

import icu
from collections import Counter
from math import exp, log


def norm(s):
  trans = str.maketrans('ς' + 'ךםןףץ' + '٩٨٧٦٥٤٣٢١٠', 'σ' + 'כמנפצ' + '9876543210')
  return s.translate(trans).lower()


def perplexity(cnt):
  s = sum(cnt.values())
  return exp(-sum(log(c / s) * c for c in cnt.values()) / s)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.1/268.1 kB 11.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyICU: filename=pyicu-2.16-cp312-cp312-linux_x86_64.whl size=2721123 sha256=d398d8d6db47b79f98ffd16643ba61df76eabb914de6915d9a870bef8a314b1f
  Stored in directory: /root/.cache/pip/wheels/d8/d3/8e/efa19e1de2f9cf1451baf994a55dddcaa41a94185f2501ca08
Successfully built PyICU


# TED2020

    @inproceedings{reimers-2020-multilingual-sentence-bert,
      title = "Making Monolingual Sentence Embeddings Multilingual using Knowledge Distillation",
      author = "Reimers, Nils and Gurevych, Iryna",
      booktitle = "Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing",
      month = "11",
      year = "2020",
      publisher = "Association for Computational Linguistics",
      url = "https://arxiv.org/abs/2004.09813",
    }

https://arxiv.org/abs/2004.09813

https://huggingface.co/datasets/wecover/OPUS_TED2020

In [2]:
from datasets import load_dataset


ds = load_dataset('wecover/OPUS_TED2020', 'he')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/2723 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2723 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/2723 [00:00<?, ?it/s]

...

TED2020/he-uz/valid.parquet:   0%|          | 0.00/88.4k [00:00<?, ?B/s]

TED2020/he-vi/valid.parquet:   0%|          | 0.00/402k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/976725 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/122110 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/122090 [00:00<?, ? examples/s]

In [19]:
bi = {}

for _, rows in ds.items():
  for row in rows:
    laaz = (row['lang1'], row['sentence1'])
    heb = (row['lang2'], row['sentence2'])
    if laaz[0] == 'he':
      laaz, heb = heb, laaz
    assert heb[0] == 'he', row
    lang = laaz[0]
    if lang not in bi:
      bi[lang] = [0, 0, 0, Counter(), Counter()]
    bi[lang][0] += len(laaz[1])
    bi[lang][1] += len(heb[1])
    bi[lang][2] += 1
    bi[lang][3].update(set(norm(laaz[1])))
    bi[lang][4].update(set(norm(heb[1])))

ted1 = sorted([(f'{icu.Locale(k).getDisplayName()} ({k})', v[0] / v[1], v[2], len(v[3]), perplexity(v[3]), perplexity(v[4])) for k, v in bi.items()], key=lambda x: (x[1], x[0]))
for lang, rel, lines, chars, perp, pheb in ted1:
  print(f'{lang}\t{round(rel, 3):.3f} ({lines}) {chars} {perp:.1f} {round(perp / pheb * rel, 3):.3f}')

Japanese (ja)	0.545 (25000) 2637 318.0 5.981
Korean (ko)	0.698 (25000) 1696 262.1 6.296
Amharic (am)	0.786 (1023) 279 111.7 3.031
Latin (la)	0.995 (20) 30 24.8 0.880
Arabic (ar)	1.092 (25000) 239 41.0 1.546
Pashto (ps)	1.166 (962) 104 41.0 1.692
Kazakh (kk)	1.177 (9484) 115 39.3 1.601
Polish (pl)	1.183 (25000) 115 35.8 1.462
Thai (th)	1.194 (25000) 151 55.1 2.274
Slovenian (sl)	1.196 (25000) 100 30.0 1.234
Czech (cs)	1.200 (25000) 107 38.8 1.613
Assamese (as)	1.212 (41) 65 47.2 2.234
Serbian (sr)	1.216 (25000) 118 46.7 1.960
Marathi (mr)	1.217 (20287) 162 50.1 2.137
Slovak (sk)	1.217 (25000) 105 40.1 1.688
Bosnian (bs)	1.234 (11058) 79 31.1 1.335
Croatian (hr)	1.236 (25000) 98 31.6 1.345
Esperanto (eo)	1.236 (11079) 81 31.2 1.328
Ukrainian (uk)	1.240 (25000) 125 39.3 1.687
Mongolian (mn)	1.240 (23778) 115 36.2 1.552
Azerbaijani (az)	1.241 (11402) 79 36.9 1.577
Nepali (ne)	1.243 (4112) 139 53.2 2.288
Belarusian (be)	1.245 (10036) 111 39.3 1.695
Finnish (fi)	1.246 (25000) 96 27.3 1.179
E

# TED-Parallel-Corpus

    @techreport{kulkarni:hal-04702210,
      TITLE = {{TED-Parallel-Corpus}},
      AUTHOR = {Kulkarni, Ajinkya},
      URL = {https://hal.science/hal-04702210},
      NUMBER = {none},
      INSTITUTION = {{CDAC ; Idiap Research Institure}},
      YEAR = {2015},
      MONTH = Jan,
      KEYWORDS = {Machine translation ; Neural machine translation NMT ; NLP ; Multilingual},
      PDF = {https://hal.science/hal-04702210v1/file/ted_parallel_corpus.pdf},
      HAL_ID = {hal-04702210},
      HAL_VERSION = {v1},
    }

https://hal.science/hal-04702210

https://github.com/ajinkyakulkarni14/How-I-Extracted-TED-talks-for-parallel-Corpus-

https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/

In [4]:
!wget https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/raw/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip -O Multi_lingual_Parallel_corpus_1.zip
!unzip -o Multi_lingual_Parallel_corpus_1.zip

--2025-12-01 02:46:57--  https://github.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/raw/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip [following]
--2025-12-01 02:46:58--  https://raw.githubusercontent.com/ajinkyakulkarni14/TED-Multilingual-Parallel-Corpus/refs/heads/master/Multilingual_Parallel_Corpus/Multi_lingual_Parallel_corpus_1.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response

In [22]:
import re


last = 0
lines = 0
seen = set()
multi = {}
counters = {}


with open(r'Multilingual_Parllel_corpus.txt', encoding='utf8') as f:
  for line in f.readlines():
    line = line.strip()
    if re.match(r'\d+:[a-z]{2}(-[a-z]{2})?:', line):
      num = int(line.split(':', 1)[0])
      if num != last:
        assert not seen or len(seen) == 9, (last, seen)
        seen = set()
        last = num
        lines += 1
      parts = line.split(':', 2)
      lang = parts[1]
      if lang in seen:
        continue
      seen.add(lang)
      if lang not in multi:
        multi[lang] = 0
        counters[lang] = Counter()
      text = parts[2]
    else:
      text = ' ' + line
    multi[lang] += len(text)
    counters[lang].update(set(norm(text)))

print(f'{lines=}')
ted2 = sorted([(f'{icu.Locale(k).getDisplayName()} ({k})', v / multi['he'], len(counters[k]), perplexity(counters[k])) for k, v in multi.items()], key=lambda x: (x[1], x[0]))
pheb = perplexity(counters['he'])
for lang, rel, chars, perp in ted2:
  print(f'{lang}\t{round(rel, 3):.3f} {chars} {perp:.1f} {round(perp / pheb * rel, 3):.3f}')


lines=349049
Hebrew (he)	1.000 125 26.1 1.000
Arabic (ar)	1.104 275 35.6 1.508
Russian (ru)	1.322 153 33.8 1.715
Dutch (nl)	1.347 129 27.3 1.411
Spanish (es)	1.372 175 28.6 1.507
Portuguese (Brazil) (pt-br)	1.377 133 30.0 1.586
Italian (it)	1.399 126 26.9 1.443
German (de)	1.475 140 28.4 1.609
French (fr)	1.512 154 30.1 1.748
